[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [1]:
from transformers import pipeline
import torch
import torch.nn.functional as F

ModuleNotFoundError: No module named 'transformers'

In [1]:
classifier = pipeline("text_classification")

NameError: name 'pipeline' is not defined

In [ ]:
# Load Huggingface transformers


# # Then what you need from tensorflow.keras
# from tensorflow.keras.layers import Input, Dropout, Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.initializers import TruncatedNormal
# from tensorflow.keras.losses import CategoricalCrossentropy
# from tensorflow.keras.metrics import CategoricalAccuracy
# from tensorflow.keras.utils import to_categorical



In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Import data from csv
df = pd.read_csv('./data/MBTI 500.csv')


# Remove a row if any of the three remaining columns are missing
df = df.dropna()


# Set your model output as categorical and save in new label col
df['type_cat'] = pd.Categorical(df['type'])

# Transform your output to numeric
df['type_cat'] = df['type_cat'].cat.codes

# Split into train and test - stratify over Issue
df, df_test = train_test_split(df, test_size = 0.2, stratify = df[['type']])

print(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84853 entries, 83315 to 72475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   posts     84853 non-null  object
 1   type      84853 non-null  object
 2   type_cat  84853 non-null  int8  
dtypes: int8(1), object(2)
memory usage: 2.0+ MB
None


,posts,type,type_cat
83315,well psych grad student love stuff probably pa...,INFJ,8
88495,peer pressure always respond certainly take em...,INFJ,8
95666,want talk two ex past say need stop try therap...,INFP,9


In [7]:
from transformers import TFBertModel, BertConfig, BertTokenizerFast

# Name of the BERT model to use
model_name = "bert-base-uncased"
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Downloading: 100%|██████████| 511M/511M [00:13<00:00, 40.6MB/s] 
2022-02-28 17:44:38.065763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-28 17:44:38.066134: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [12]:
# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model

# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
types = Dense(units=len(df.type_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='type')(pooled_output)
outputs = {'type': types}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 100)]             0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  109482240 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             100, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                        

In [13]:
#######################################
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = {'type': CategoricalCrossentropy(from_logits = True)}
metric = {'type': CategoricalAccuracy('accuracy')}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
# Ready output data for the model
y_type = to_categorical(df['type_cat'])
# Tokenize the input (takes some time)
x = tokenizer(
    text=df['posts'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'issue': y_type},
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Ignored unknown kwarg option direction
Epoch 1/10


ValueError: in user code:

    File "/Users/westonshuken/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/westonshuken/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/westonshuken/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/westonshuken/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 864, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/westonshuken/tensorflow-test/env/lib/python3.8/site-packages/keras/engine/training.py", line 957, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/U